# Load Packages

In [ ]:
%load_ext autoreload
%autoreload 2

import time
from matplotlib import pyplot as plt
from threading import Thread
import RT_functions as rt
import RT_arduino as arduino

# Initialize connection with Arduino

In [ ]:
ard = arduino.Arduino(
    port = '/dev/cu.usbmodem14401',
    keys_dout = {
        'light' : 13,
        'ttl' : 7
    }
)
ard.initialize()

# Create Video Instance

In [ ]:
vid = rt.Video(src=0)
vid.start()

# Define Cropping Bounds

In [ ]:
%output size = 50

vid.crop_define()

# Select Regions of Interest

In [ ]:
%output size = 100

vid.roi_define(names=['region'])

# Set Reference Frame for Tracking

In [ ]:
vid.ref_create(secs=1, print_sts=False)

plt.imshow(vid.ref, cmap='gray')

# Set Tracking Parameters

In [ ]:
vid.track_method = 'abs'
vid.track_thresh = 95
vid.track_window_use = False
vid.track_window_sz = 100
vid.track_window_wt = 0.9
vid.track_rmvwire = False
vid.track_rmvwire_krn = 10

# Initialize tracking

In [ ]:
vid.track = True

# Initialize Data Writer
This should be done after any cropping is performed and tracking has begun (if tracking), as a video instance of the current frame size will be initiated.

In [ ]:
vid.writer_init(dpath='/Users/zachpennington/Desktop/test')

# Run task

`Video.display` cannot be run within its own thread due to OpenCV constraints.  As a result, we start the task in a thread and then open displayer.

In [ ]:
def task(): 
    duration = 10
    in_region = False
    vid.writer_start() 
    start_time = time.time() 
    while time.time()-start_time < duration:
        if vid.track_roi['region']:
            if not in_region:
                in_region = True
                ard.digitalHigh('light')
        else:
            if in_region:
                in_region = False
                ard.digitalLow('light')
        time.sleep(.001) #Allow other threads to run
    vid.release()

In [ ]:
Thread(target=task, args=()).start()  
vid.display(show_xy=True, show_dif=True)  